In [1]:

def optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter=None, save_iter=None):
    def iterate():
        t[0] += 1
    
        optimizer.zero_grad()
        stylenet(img)
        loss = stylenet.get_loss()
        loss.backward()
        
        maybe_print(stylenet, t[0], print_iter, num_iterations, loss)
        maybe_save(img, t[0], save_iter, num_iterations, original_colors, output_path)

        return loss
    
    img = nn.Parameter(img.type(stylenet.dtype))
    optimizer, loopVal = setup_optimizer(img, stylenet.params, num_iterations)
    t = [0]
    while t[0] <= loopVal:
        optimizer.step(iterate)
    
    return img
    
from model import *
from utils import *

# setup stylenet
params = StylenetArgs()
params.gpu = '0'
params.backend = 'cudnn'

dtype, multidevice, backward_device = setup_gpu(params)
stylenet = StyleNet(params, dtype, multidevice, backward_device)




VGG-19 Architecture Detected
Successfully loaded models/vgg19-d01eb7cb.pth
conv1_1: 64 3 3 3
conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3
conv3_3: 256 256 3 3
conv3_4: 256 256 3 3
conv4_1: 512 256 3 3
conv4_2: 512 512 3 3
conv4_3: 512 512 3 3
conv4_4: 512 512 3 3
conv5_1: 512 512 3 3
conv5_2: 512 512 3 3
conv5_3: 512 512 3 3
conv5_4: 512 512 3 3
Setting up style layer 2: relu1_1
Setting up style layer 7: relu2_1
Setting up style layer 12: relu3_1
Setting up style layer 21: relu4_1
Setting up content layer 23: relu4_2
Setting up style layer 30: relu5_1
Sequential(
  (0): TVLoss()
  (1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): ReLU(inplace=True)
  (3): MaskedStyleLoss(
    (gram): GramMatrix()
    (crit): MSELoss()
  )
  (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (

In [ ]:
image_size = 512
num_iterations = 500
output_path = 'out.png'

original_colors = 0
style_scale = 1.0
print_iter = 100
save_iter = 100

content_path = 'examples/inputs/monalisa.jpg'
style_paths = ['examples/inputs/starry_night.jpg','examples/inputs/cubist.jpg']

# load content image
content_image = load_image(content_path, image_size)

# load style images
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size) for path in style_paths]

# set hyper-parameters
stylenet.set_content_weight(5e0)
stylenet.set_style_weight(1e2)
stylenet.set_tv_weight(1e-3)

# capture the style and content images
stylenet.capture(content_image, style_images)

# initialize with a random image
img = random_image_like(content_image)

optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter, save_iter)

In [ ]:
        
image_size = 512
num_iterations = 500
output_path = 'out2.png'
original_colors = 0
style_scale = 1.0
print_iter = 100
save_iter = 100


content_path = 'examples/inputs/monalisa.jpg'
style_paths = ['examples/inputs/starry_night.jpg','examples/inputs/cubist.jpg']
content_mask_paths = ['examples/segments/monalisa1a.png','examples/segments/monalisa1b.png']

# load content image
content_image = load_image(content_path, image_size)#.type(dtype)

# load style images
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size) 
                for path in style_paths]

# load masks
content_masks = [load_image(path, image_size, to_normalize=False) 
                 for path in content_mask_paths]

# capture the style and content images (with masks)
stylenet.capture(content_image, style_images, None, content_masks)
            
# set hyper-parameters
stylenet.set_content_weight(5e0)
stylenet.set_style_weight(1e2)
stylenet.set_tv_weight(1e-3)

# initialize with a random image
img = random_image_like(content_image)

optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter, save_iter)

In [2]:

#######################################################3


image_size = 1280
num_iterations = 1200
output_path = 'go5.png'

original_colors = 0
style_scale = 0.25
print_iter = 100
save_iter = 100

content_path = 'examples/inputs/hoovertowernight.jpg'


style_paths = ['examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg',
                'examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg',
                'examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg',
                'examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg',
                'examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg',
                'examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg']


style_blend_weights = [1.0, 0.0,
                      0.8, 0.2,
                       0.6, 0.4,
                       0.4, 0.6,
                       0.2, 0.8,                      
                      0.0, 1.0]

# load content image
content_image = load_image(content_path, image_size)

# load style images
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size) for path in style_paths]

stylenet.set_content_weight(0)
stylenet.set_style_weight(1e2)
stylenet.set_tv_weight(1e-3)

# capture the style and content images

# initialize with a random image
img = random_image_like(content_image)



content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
m=8
content_masks[0][:,:,:,:360+m] = 1.0
content_masks[1][:,:,:,:360+m] = 1.0
content_masks[2][:,:,:,360-m:500+m] = 1.0
content_masks[3][:,:,:,360-m:500+m] = 1.0
content_masks[4][:,:,:,500-m:640+m] = 1.0
content_masks[5][:,:,:,500-m:640+m] = 1.0
content_masks[6][:,:,:,640-m:780+m] = 1.0
content_masks[7][:,:,:,640-m:780+m] = 1.0
content_masks[8][:,:,:,780-m:920+m] = 1.0
content_masks[9][:,:,:,780-m:920+m] = 1.0
content_masks[10][:,:,:,920-m:] = 1.0
content_masks[11][:,:,:,920-m:] = 1.0


stylenet.capture(content_image, style_images, style_blend_weights, content_masks)

optimize(stylenet, img, 1000, output_path, original_colors, print_iter, save_iter)

Iteration 1000/1000: 
  Content loss = 0.0e+00
  Style loss = 2.4e+04, 3.1e+06, 1.2e+06, 4.0e+07, 3.0e+03
  TV loss = 3.5e+04
  Total loss = 4.39e+07


Parameter containing:
tensor([[[[ 107.4170,  107.3822,  107.2753,  ...,   18.5964,   18.6580,
             18.6587],
          [ 107.4086,  107.3838,  107.2558,  ...,   18.5873,   18.6362,
             18.6319],
          [   6.0468,    6.0569,   26.0255,  ...,   18.5962,   18.6259,
             18.6241],
          ...,
          [ -15.5395,  -15.5464,  -15.5937,  ...,   -4.7783,   -4.8542,
             -4.8807],
          [ -15.6088,  -15.5838,  -15.5704,  ...,   -4.8293,   -4.8587,
             -4.8855],
          [ -15.5850,  -15.5833,  -15.5725,  ...,   -4.8222,   -4.8541,
             -4.8611]],

         [[  20.6844,   20.6578,   20.5571,  ...,   20.3936,   20.4353,
             20.4430],
          [  20.6818,   20.6415,   20.5252,  ...,   20.3572,   20.4206,
             20.4426],
          [ -93.5266,  -94.0915,  -97.6999,  ...,   20.3577,   20.4144,
             20.4342],
          ...,
          [ -40.6276,  -40.6149,  -40.6026,  ...,   13.2112,   13.2194,
             13.230

In [3]:
import IPython.display
import numpy as np
import PIL.Image
from io import BytesIO 
import IPython.display
import numpy as np

def showarray(a, fmt='png'):
    a = np.uint8(np.array(a))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

x = 0
content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
content_masks[0][:,:,:,:x] = torch.linspace(0, 1, x)
content_masks[0][:,:,:,x:] = torch.linspace(1, 0, 720-x)
content_masks[1][:,:,:,:x] = torch.linspace(0, 1, x)
content_masks[1][:,:,:,x:] = torch.linspace(1, 0, 720-x)

showarray(np.transpose(np.array(255*content_masks[0][0]), [1,2,0]).astype(np.uint8))

RuntimeError: The expanded size of the tensor (1280) must match the existing size (720) at non-singleton dimension 3.  Target sizes: [1, 3, 673, 1280].  Tensor sizes: [720]

In [ ]:

#######################################################3


image_size = 720
num_iterations = 1200
output_path = 'go6.png'

original_colors = 0
style_scale = 1.0
print_iter = 100
save_iter = 100

content_path = 'examples/inputs/hoovertowernight.jpg'
style_paths = ['examples/inputs/starry_night.jpg','examples/inputs/hokusai.jpg']

# load content image
content_image = load_image(content_path, image_size)

# load style images
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size) for path in style_paths]

stylenet.set_content_weight(0)
stylenet.set_style_weight(1e2)
stylenet.set_tv_weight(1e-3)

# capture the style and content images

# initialize with a random image
img = random_image_like(content_image)






content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
content_masks[0][:,:,:,:] = torch.linspace(0, 1, 720)
content_masks[1][:,:,:,:] = torch.linspace(1, 0, 720)
#content_masks[2][:,:,:,480:] = 1.0

def optimize2(stylenet, img, num_iterations, output_path, original_colors, print_iter=None, save_iter=None):
    def iterate():
        t[0] += 1
    
        optimizer.zero_grad()
        stylenet(img)
        loss = stylenet.get_loss()
        loss.backward()
        
        maybe_print(stylenet, t[0], print_iter, num_iterations, loss)
        maybe_save(img, t[0], save_iter, num_iterations, original_colors, output_path)

        return loss
    
    img = nn.Parameter(img.type(stylenet.dtype))
    optimizer, loopVal = setup_optimizer(img, stylenet.params, num_iterations)
    t = [0]
    while t[0] <= loopVal:
        optimizer.step(iterate)
    
    return img



for z in range(8):
    for x in range(0,720,3):
        if z%2==0:
            content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
            content_masks[0][:,:,:,:x] = torch.linspace(0, 1, x)
            content_masks[0][:,:,:,x:] = torch.linspace(1, 0, 720-x)
            content_masks[1][:,:,:,:x] = torch.linspace(0, 1, x)
            content_masks[1][:,:,:,x:] = torch.linspace(1, 0, 720-x)
            bw = x/719.0;
            style_blend_weights = [bw, 1.0-bw]
        else:
            content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
            content_masks[0][:,:,:,:720-x] = torch.linspace(0, 1, x)
            content_masks[0][:,:,:,720-x:] = torch.linspace(1, 0, 720-x)
            content_masks[1][:,:,:,:720-x] = torch.linspace(0, 1, x)
            content_masks[1][:,:,:,720-x:] = torch.linspace(1, 0, 720-x)
            bw = (719.0-x)/719.0;
            style_blend_weights = [bw, 1.0-bw]

        stylenet.capture(content_image, style_images, None, content_masks)

        img = optimize2(stylenet, img, 5, output_path, original_colors, print_iter, save_iter)

In [ ]:

def optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter=None, save_iter=None):
    def iterate():
        t[0] += 1
    
        optimizer.zero_grad()
        stylenet(img)
        loss = stylenet.get_loss()
        loss.backward()
        
        maybe_print(stylenet, t[0], print_iter, num_iterations, loss)
        maybe_save(img, t[0], save_iter, num_iterations, original_colors, output_path)

        return loss
    
    img = nn.Parameter(img.type(stylenet.dtype))
    optimizer, loopVal = setup_optimizer(img, stylenet.params, num_iterations)
    t = [0]
    while t[0] <= loopVal:
        optimizer.step(iterate)
    

        
image_size = 720
num_iterations = 1000
output_path = 'try9d.png'
original_colors = 0
style_scale = 1.0
print_iter = 100
save_iter = 100


content_path = 'examples/inputs/hoovertowernight.jpg'
style_paths = ['examples/inputs/starry_night.jpg', 'examples/inputs/hokusai1.jpg']

# load content image
content_image = load_image(content_path, image_size)#.type(dtype)

# load style images
style_size = int(image_size * style_scale)
style_images = [load_image(path, style_size) 
                for path in style_paths]

#style_images[1] = torch.cat([style_images[0], style_images[2]], axis=2)

# load masks
content_masks = [torch.zeros(content_image.shape) for c in range(len(style_images))]
content_masks[0][:,:,:,:] = torch.linspace(0, 1, 720)
content_masks[1][:,:,:,:] = torch.linspace(1, 0, 720)
#content_masks[2][:,:,:,480:] = 1.0

# capture the style and content images (with masks)
stylenet.capture(content_image, style_images, None, content_masks)
            
# set hyper-parameters
stylenet.set_content_weight(0)
stylenet.set_style_weight(1e2)
stylenet.set_tv_weight(1e-3)

# initialize with a random image
img = random_image_like(content_image)

optimize(stylenet, img, num_iterations, output_path, original_colors, print_iter, save_iter)